In [10]:
# K 폴드

from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
import numpy as np

iris = load_iris()
features = iris.data
label = iris.target
df_clf = DecisionTreeClassifier(random_state=156)

kfold = KFold(n_splits=5)
print(features.shape[0]) # 전체 feature row

150


In [11]:

n_iter = 0
cv_accuracy = [] # accuracy의 값을 append 하는 list. 이후 kfold accuracy의 평균을 구하는데 사용 됨
for train_index, test_index in kfold.split(features):
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = label[train_index], label[test_index]

    df_clf.fit(X_train, y_train)
    pred = df_clf.predict(X_test)
    n_iter += 1

    accuracy = np.round(accuracy_score(y_test, pred), 4)
    train_size = X_train.shape[0]
    test_size = X_test.shape[0]

    cv_accuracy.append(accuracy)

print(cv_accuracy)
print(np.mean(cv_accuracy))
print((sum(cv_accuracy))/n_iter)

[1.0, 0.9667, 0.8667, 0.9333, 0.7333]
0.9
0.9


In [21]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=3)

n_iter = 0
cv_accuracy = []

for train_idx, test_idx in skf.split(features, label):
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = label[train_index], label[test_index]

    
    print(y_test)

    df_clf.fit(X_train, y_train)
    pred = df_clf.predict(X_test)
    n_iter += 1

    accuracy = np.round(accuracy_score(y_test, pred), 4)
    train_size = X_train.shape[0]
    test_size = X_test.shape[0]

    cv_accuracy.append(accuracy)

print(cv_accuracy)
print(np.mean(cv_accuracy))
print((sum(cv_accuracy))/n_iter)


[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
[0.7333, 0.7333, 0.7333]
0.7333
0.7333
